In [1]:
import os, shutil, itertools, yaml, kornia, torchvision, sys, copy, math
from functools import partial
import dill as pickle
from importlib import reload
osp = os.path
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms
import seaborn as sns
from PIL import Image
import torch
nn = torch.nn
F = nn.functional
import monai.transforms as mtr
from time import time

In [2]:
from inrnet import args as args_module
from inrnet import inn, experiments, optim, util, losses, models, jobs as job_mgmt
from inrnet.data import dataloader
from inrnet.experiments import depth
import inrnet.inn.functional as inrF
%matplotlib inline
plt.rcParams["figure.figsize"] = (4.0, 3.0)
plt.rcParams['figure.dpi'] = 200

rescale_clip = mtr.ScaleIntensityRangePercentiles(lower=1, upper=99, b_min=0, b_max=255, clip=True, dtype=np.uint8)
rescale_noclip = mtr.ScaleIntensityRangePercentiles(lower=0, upper=100, b_min=0, b_max=255, clip=False, dtype=np.uint8)
rescale_float = mtr.ScaleIntensity()

TMP_DIR=osp.expanduser("~/code/diffcoord/temp")

rsync -av --ignore-existing \
clintonw@peppercorn.csail.mit.edu:/data/vision/polina/users/clintonw/code/placenta/results/unetr_00 Downloads/

# interactive

In [ ]:
bash train.sh inet_nn_train inet_nn_train
bash train.sh inet_inn_train inet_train
bash train.sh inet_scr_train inet_scr_train

bash infer.sh i1x_nn inet_val inet_nn_train
bash infer.sh i1x_inn inet_val inet_inn_train
bash infer.sh i1x_scr inet_val inet_scr_train
bash infer.sh i2x_nn in2x_val inet_nn_train
bash infer.sh i2x_inn in2x_val inet_inn_train
bash infer.sh i2x_scr in2x_val inet_scr_train
bash infer.sh ihx_nn inhx_val inet_nn_train
bash infer.sh ihx_inn inhx_val inet_inn_train
bash infer.sh ihx_scr inhx_val inet_scr_train
python infer.py -j=inet_inn_val -c=inet_val -t=inet_inn_train

In [ ]:
bash infer.sh i1x_inn inet_val inet_inn_rand
bash infer.sh i1x_scr inet_val inet_scr_rand
bash infer.sh i2x_inn in2x_val inet_inn_rand
bash infer.sh i2x_scr in2x_val inet_scr_rand
bash infer.sh ihx_inn inhx_val inet_inn_rand
bash infer.sh ihx_scr inhx_val inet_scr_rand

In [6]:
sd = torch.load('/data/vision/polina/users/clintonw/code/diffcoord/temp/upernet_convnext.pth')['state_dict']

In [42]:
root = 'decode_head.fpn_bottleneck.'
fpn_bot_sd = {k[len(root):]:v for k,v in sd.items() if k.startswith(root)}

In [ ]:
fpn_bot_sd.keys()

In [ ]:
input_shape = (32,32)

In [79]:
root = 'decode_head.fpn_bottleneck.'
fpn_bot_sd = {k[len(root):]:v for k,v in sd.items() if k.startswith(root)}

In [ ]:
torch.load("/data/vision/polina/users/clintonw/code/diffcoord/results/inet/results.txt")

In [ ]:
CUDA_VISIBLE_DEVICES=3 python train.py -j=inet2 -c=inet

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [97]:
base = torch.load(osp.expanduser('~/code/diffcoord/temp/upernet_convnext.pth'))

In [203]:
img = plt.imread('/data/vision/polina/scratch/clintonw/datasets/cityscapes/gtCoarse/val/frankfurt/frankfurt_000001_080091_gtCoarse_labelIds.png')

In [ ]:
base = torchvision.models.efficientnet_b0(pretrained=True)
base

In [42]:
DS_DIR = "/data/vision/polina/scratch/clintonw/datasets"
ds = torchvision.datasets.CIFAR10(root=DS_DIR, transform=transforms.ToTensor())

In [78]:
torch.cuda.empty_cache()
coords = util.meshgrid_coords(64,64)
#spacing = torch.tensor((4/127,4/127), device="cuda")
values = torch.randn(coords.size(0),6, device=coords.device)

In [ ]:
ax = sns.lineplot(x=torch.linspace(128,256,10), y=torch.linspace(.5,.8,10)+torch.randn(10)/8, label="qmc");
sns.lineplot(x=torch.linspace(128,256,10), y=torch.linspace(.1,.8,10)+torch.randn(10)/8, label="grid", ax=ax);
ax.set_xlabel("Number of sample points");
ax.set_ylabel("ImageNet top-1 error");

In [67]:
## search python strings
import os
osp = os.path
root = osp.expanduser("~/code/diffcoord/inrnet")
query_string = 'first_split_meshgrid'
for folder, subfolders, files in os.walk(root):
    for file in files:
        if file.endswith(".py"):
            path = osp.join(folder, file)
            with open(path, 'r') as f:
                if query_string in f.read():
                    print(path)

/data/vision/polina/users/clintonw/code/diffcoord/inrnet/util.py


## inet12

In [ ]:
from robustness.tools.imagenet_helpers import ImageNetHierarchy, common_superclass_wnid
in_path = '/data/vision/polina/scratch/clintonw/datasets/imagenet_pytorch'
in_hier = ImageNetHierarchy(in_path, in_path)
superclass_wnid = common_superclass_wnid('big_12')
class_ranges, label_map = in_hier.get_subclasses(superclass_wnid,
                                                 balanced=True)

sub_to_super = {}
for supercls,subclasses in enumerate(class_ranges):
    for subcls in subclasses:
        sub_to_super[subcls] = supercls

big12path = f"{DS_DIR}/inrnet/big_12.pkl"
pickle.dump((label_map, class_ranges, sub_to_super), open(big12path, 'wb'))

label_to_super = {}
for supercls,subclasses in enumerate(class_labels):
    for subcls in subclasses:
        label_to_super[subcls] = supercls

big12path = f"{DS_DIR}/inrnet/big_12_labels.pkl"
pickle.dump((split_paths_by_cls, class_labels, label_to_super), open(big12path, 'wb'))

label = osp.basename(osp.dirname(subpaths[0]))

subpaths_by_cls = [[] for _ in range(12)]
for path in subpaths:
    label = osp.basename(osp.dirname(path))
    if label in label_to_super.keys():
        subpaths_by_cls[label_to_super[label]].append(path)

split_paths_by_cls = {'train':[p[:800] for p in subpaths_by_cls],
                     'test':[p[800:] for p in subpaths_by_cls]}

for p in subpaths_by_cls:
    np.random.shuffle(p)

subpaths = open("/data/vision/polina/scratch/clintonw/datasets/imagenet_pytorch/val.txt",
            "r").read().split('\n')

labels = open("/data/vision/polina/scratch/clintonw/datasets/imagenet_pytorch/labels.txt",
            "r").read().split('\n')

labels = [l[:l.find(',')] for l in labels[:-1]]

class_labels = [np.array(labels)[list(cr)].tolist() for cr in class_ranges]

paths = util.glob2('/data/vision/polina/scratch/clintonw/datasets/inrnet/cityscapes/train_*.pt')
for p in paths:
    weights, seg = torch.load(p)
    break

from inrnet.models.inrs import siren

keys = siren.get_siren_keys()

# Tables

### Classify

In [77]:
RESULTS_DIR = osp.expanduser("~/code/diffcoord/results")

model_jobs = {'EffNet-T':('i1x_nn', 'i2x_nn', 'ihx_nn'),
    'INR-tuned (ours)':('i1x_inn', 'i2x_inn', 'ihx_inn'),
    'INR-scratch (ours)':('i1x_scr', 'i2x_scr', 'ihx_scr')}
N = 200*12
for model,row in model_jobs.items():
    rowstr = [model]
    for job in row:
        path = osp.join(RESULTS_DIR, job, "stats.pt")
        if not osp.exists(path):
            print(f'missing {job} results at {path}')
            rowstr.append("nan")
        else:
            top1, top3 = torch.load(path)
            rowstr.append(util.format_float(top3/N*100, n_decimals=1)+r"\%")
    print(' & '.join(map(str,rowstr)) + r' \\')

EffNet-T & 67.3\% & 64.9\% & 56.6\% \\
INR-tuned (ours) & 31.8\% & 25.7\% & 24.2\% \\
INR-scratch (ours) & 39.0\% & 28.5\% & 25.0\% \\


In [ ]:
jobs = ["raw_m_rgae", "dit_m_rgae", "raw_m_caae", "dit_m_caae",
        "raw_m_ipg", "dit_m_ipg", "raw_m_star", "dms_1"]
df = tables.get_results_table()
subtable = df.loc[jobs]

def latex_format(value, n_decimals=2, bold=False):
    if bold:
        return (r'$\bm{{{0:.%df}}}$'%n_decimals).format(value)        
    else:
        return (r'${0:.%df}$'%n_decimals).format(value)
def plus_minus_format(mean, std, n_decimals=2, bold=False):
    if bold:
        return (r'$\bm{{{0:.%df}}}$\scriptsize$\bm{{\pm {1:.%df}}}$'%(n_decimals, n_decimals)).format(mean, std)        
    else:
        return (r'${0:.%df}$\scriptsize$\pm {1:.%df}$'%(n_decimals, n_decimals)).format(mean, std)

def print_row(name, row, bold=()):
    field1 = latex_format(row["FID_tuned"], bold=1 in bold)
    field2 = latex_format(row["F_{1/8}"], bold=2 in bold)
    field3 = latex_format(row["F_8"], bold=3 in bold)
    field4 = plus_minus_format(row["age mean error"], row["age error STD"], bold=4 in bold)
    print(" & ".join([name, field1, field2, field3, field4]) + r" \\")

### Segment

In [ ]:
model_jobs = {'EffNet-T':('c1x_nn', 'c2x_nn', 'chx_nn'),
    'INR-tuned (ours)':('c1x_inn', 'c2x_inn', 'chx_inn'),
    'INR-scratch (ours)':('c1x_scr', 'c2x_scr', 'chx_scr')}
N = 200*12
for model,row in model_jobs.items():
    rowstr = [model]
    for job in row:
        path = osp.join(RESULTS_DIR, job, "stats.pt")
        if not osp.exists(path):
            print(f'missing {job} results at {path}')
            rowstr.append("nan")
        else:
            top1, top3 = torch.load(path)
            rowstr.append(util.format_float(top3/N*100, n_decimals=1)+r"\%")
    print(' & '.join(map(str,rowstr)) + r' \\')

### Generate

In [ ]:
model_jobs = {'EffNet-T':('f1x_nn', 'f2x_nn', 'fhx_nn'),
    'INR-tuned (ours)':('f1x_inn', 'f2x_inn', 'fhx_inn'),
    'INR-scratch (ours)':('f1x_scr', 'f2x_scr', 'fhx_scr')}
N = 200*12
for model,row in model_jobs.items():
    rowstr = [model]
    for job in row:
        path = osp.join(RESULTS_DIR, job, "stats.pt")
        if not osp.exists(path):
            print(f'missing {job} results at {path}')
            rowstr.append("nan")
        else:
            top1, top3 = torch.load(path)
            rowstr.append(util.format_float(top3/N*100, n_decimals=1)+r"\%")
    print(' & '.join(map(str,rowstr)) + r' \\')

# Figs

In [ ]:
xy = util.generate_quasirandom_sequence(d=2, n=256)
plt.rcParams["figure.figsize"] = (2,1)
fig,ax = plt.subplots()
ax.scatter(xy[:,0], xy[:,1], s=2, c='k')
ax.plot();

In [ ]:
kernel = model.features[0][0].weight[3,1].detach()
plt.rcParams["figure.figsize"] = (.5,.5)
fig,ax = plt.subplots()
print(kernel.min().item(), kernel.max().item())
ax.imshow(kernel, vmin=-2.9, vmax=2.9);
plt.axis('off');

In [100]:
from scipy.interpolate import RectBivariateSpline as Spline2D
K = [2.5,2.5]
h,w=3,3
bbox = (-K[0]/2, K[0]/2, -K[1]/2, K[1]/2)
x,y = (np.linspace(bbox[0]/h*(h-1), bbox[1]/h*(h-1), h),
       np.linspace(bbox[2]/w*(w-1), bbox[3]/w*(w-1), w))
bs = Spline2D(x,y, kernel, bbox=bbox, kx=2,ky=2, s=0)
tx,ty,c = [torch.tensor(z).float() for z in bs.tck]
c = c.reshape(h,w)

In [106]:
H = 50
xy = util.meshgrid_coords(H,H).cpu()
w_oi = []
X = xy[:,0].unsqueeze(1)
Y = xy[:,1].unsqueeze(1)
px = py = 2

values, kx = (tx<=X).min(dim=-1)
values, ky = (ty<=Y).min(dim=-1)
kx -= 1
ky -= 1
kx[values] = tx.size(-1)-px-2
ky[values] = ty.size(-1)-py-2

Ctrl = c.view(1, *c.shape[-2:])
for z in range(X.size(0)):
    D = Ctrl[:, kx[z]-px : kx[z]+1, ky[z]-py : ky[z]+1].clone()

    for r in range(1, px + 1):
        try:
            alphax = (X[z,0] - tx[kx[z]-px+1:kx[z]+1]) / (
                tx[2+kx[z]-r:2+kx[z]-r+px] - tx[kx[z]-px+1:kx[z]+1])
        except RuntimeError:
            print("input off the grid")
        for j in range(px, r - 1, -1):
            D[:,j] = (1-alphax[j-1]) * D[:,j-1] + alphax[j-1] * D[:,j].clone()

    for r in range(1, py + 1):
        alphay = (Y[z,0] - ty[ky[z]-py+1:ky[z]+1]) / (
            ty[2+ky[z]-r:2+ky[z]-r+py] - ty[ky[z]-py+1:ky[z]+1])
        for j in range(py, r-1, -1):
            D[:,px,j] = (1-alphay[j-1]) * D[:,px,j-1].clone() + alphay[j-1] * D[:,px,j].clone()

    w_oi.append(D[:,px,py])

w = torch.stack(w_oi).view(xy.size(0))

In [107]:
k = w.reshape(H,H)

In [ ]:
fig,ax = plt.subplots()
print(k.min().item(), k.max().item())
ax.imshow(k, vmin=-2.9, vmax=2.9);
plt.axis('off');

# slurm

In [45]:
coco_len = 118287
kitti_len = 80896
horse_len = 1067
zebra_len = 1334
inet_len = 50000
in_tr_len = 50000

In [ ]:
configs/convnext/upernet_convnext_tiny
configs/_base_/models/upernet_convnext.py

In [ ]:
for ix in range(0,places_len,800):
    if not osp.exists(f"/data/vision/polina/scratch/clintonw/datasets/inrnet/places/siren_{ix+63}.pt"):
        print("sh fit_inr.sh", f"place{ix//800}", "fit_places", ix)

In [ ]:
for ix in range(0,in_tr_len,800):
    if not osp.exists(f"/data/vision/polina/scratch/clintonw/datasets/inrnet/imagenet1k_train/siren_{ix+63}.pt"):
        print("sh fit_inr.sh", f"inet{ix//800}", "fit_in_tr", ix)

In [86]:
for ix in range(0,12000,100):
    c,i=ix//1000, ix%1000
    if i >= 800:
        i-=800
        s = 'test'
    else:
        s='train'
    if not osp.exists(f"/data/vision/polina/scratch/clintonw/datasets/inrnet/inet12/{c}/{s}_{i+3}.pt"):
        print("sh fit_inr.sh", f"inet{ix//100}", "fit_i12", ix)

In [ ]:
for ix in range(0,8189,100):
    i = ix
    if i < 1020:
        s='train'
    elif i < 2040:
        s='val'
        i -= 1020
    else:
        s='test'
        i -= 2040
    if not osp.exists(f"/data/vision/polina/scratch/clintonw/datasets/inrnet/flowers/{s}_{i+50}.pt"):
        print("sh fit_inr.sh", f"flow{ix//100}", "fit_flower", ix)

In [ ]:
for ix in range(0,3475,100):
    i = ix
    if i >= 2975:
        i -= 2975
        s = 'val'
    else:
        s = 'train'
    if not osp.exists(f"/data/vision/polina/scratch/clintonw/datasets/inrnet/cityscapes_nonorm/{s}_{i+30}.pt"):
        print("sh fit_inr.sh", f"city{ix//100}", "fit_city", ix)

In [46]:
for ix in range(0,inet_len,800):
    if not osp.exists(f"/data/vision/polina/scratch/clintonw/datasets/inrnet/imagenet1k_val/siren_{ix+63}.pt"):
        print("sh fit_inr.sh", f"inet{ix//800}", "fit_inet", ix)

In [ ]:
for ix in range(0,coco_len,1600):
    if not osp.exists(f"/data/vision/polina/scratch/clintonw/datasets/inrnet/coco/siren_{ix+63}.pt"):
        print("sh fit_inr.sh", f"coco{ix//1600}", "fit_coco", ix)

In [ ]:
for ix in range(0,kitti_len,800):
    if not osp.exists(f"/data/vision/polina/scratch/clintonw/datasets/inrnet/kitti/siren_{ix+63}.pt"):
        print("sh fit_inr.sh", f"skit{ix//800}", "fit_kitti", ix)

In [ ]:
for ix in range(0,zebra_len,128):
    if not osp.exists(f"/data/vision/polina/scratch/clintonw/datasets/inrnet/zebra/siren_{ix+15}.pt"):
        print("sh fit_inr.sh", f"zebra{ix//128}", "fit_zebra", ix)

In [ ]:
for ix in range(0,horse_len,128):
    if not osp.exists(f"/data/vision/polina/scratch/clintonw/datasets/inrnet/horse/siren_{ix+15}.pt"):
        print("sh fit_inr.sh", f"horse{ix//128}", "fit_horse", ix)

In [ ]:
for ix in range(0,10000,5000):
    if not osp.exists(f"/data/vision/polina/scratch/clintonw/datasets/inrnet/cifar10_test/siren_{ix+999}.pt"):
        print("sh fit_inr.sh", f"cit{ix//5000}", "fit_ciftest", ix)
for ix in range(0,50000,5000):
    if not osp.exists(f"/data/vision/polina/scratch/clintonw/datasets/inrnet/cifar10_train/siren_{ix+999}.pt"):
        print("sh fit_inr.sh", f"cif{ix//5000}", "fit_cifar", ix)

In [ ]:
img = plt.imread("/data/vision/polina/scratch/clintonw/datasets/inrnet/horse2zebra/testA/n02381460_9260.jpg")

In [ ]:
"/data/vision/polina/scratch/clintonw/datasets/coco/annotations/panoptic_train2017"

In [ ]:
osp.exists(f"/data/vision/polina/users/clintonw/code/diffcoord/temp/kitti/siren_{ix+63}.pt")

In [60]:
paths = util.glob2("/data/vision/polina/scratch/clintonw/datasets/inrnet/inet12/1/loss_*")

In [ ]:
paths = util.glob2("/data/vision/polina/scratch/clintonw/datasets/inrnet/cityscapes_nonorm/loss_*")
for p in paths:
    print(open(p,'r').read())